In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark=SparkSession.builder.appName('Occupancy').getOrCreate()

# Question 1 (occupancy_sensor_data.csv)

In [22]:
df_spark = spark.read.csv('occupancy_sensor_data.csv',header=True,inferSchema=True)

In [23]:
df_spark.show(5)

+-----+-------------------+-----------+--------+-----+------+-------------------+---------+
|index|               date|Temperature|Humidity|Light|   CO2|      HumidityRatio|Occupancy|
+-----+-------------------+-----------+--------+-----+------+-------------------+---------+
|    1|2015-02-04 17:51:00|      23.18|  27.272|426.0|721.25|0.00479298817650529|        1|
|    2|2015-02-04 17:51:59|      23.15| 27.2675|429.5| 714.0|0.00478344094931065|        1|
|    3|2015-02-04 17:53:00|      23.15|  27.245|426.0| 713.5|0.00477946352442199|        1|
|    4|2015-02-04 17:54:00|      23.15|    27.2|426.0|708.25|0.00477150882608175|        1|
|    5|2015-02-04 17:55:00|       23.1|    27.2|426.0| 704.5|0.00475699293331518|        1|
+-----+-------------------+-----------+--------+-----+------+-------------------+---------+
only showing top 5 rows



# Que1.1 Find out average humidity of the room

In [24]:
print((df_spark.count(), len(df_spark.columns)))

(8143, 8)


In [25]:
df_spark.agg({'Humidity':'mean'}).show()

+------------------+
|     avg(Humidity)|
+------------------+
|25.731507286422527|
+------------------+



# Que1.2 Filter all timestamp where the temperature is greater than 20 degrees

In [26]:
time=df_spark.filter((df_spark['Temperature']>20)).select('date').show()

+-------------------+
|               date|
+-------------------+
|2015-02-04 17:51:00|
|2015-02-04 17:51:59|
|2015-02-04 17:53:00|
|2015-02-04 17:54:00|
|2015-02-04 17:55:00|
|2015-02-04 17:55:59|
|2015-02-04 17:57:00|
|2015-02-04 17:57:59|
|2015-02-04 17:58:59|
|2015-02-04 18:00:00|
|2015-02-04 18:01:00|
|2015-02-04 18:02:00|
|2015-02-04 18:03:00|
|2015-02-04 18:04:00|
|2015-02-04 18:04:59|
|2015-02-04 18:06:00|
|2015-02-04 18:07:00|
|2015-02-04 18:08:00|
|2015-02-04 18:08:59|
|2015-02-04 18:10:00|
+-------------------+
only showing top 20 rows



# Que1.3 What is the maximum and minimum temperature of the room with 1 occupant?

In [27]:
from pyspark.sql.functions import min,max 

In [28]:
df_spark.filter(df_spark['Occupancy']==1).agg(min('Temperature').alias('Min_Temp'),max('Temperature').alias('Max_Temp')).show()

+--------+--------+
|Min_Temp|Max_Temp|
+--------+--------+
|    19.5|   23.18|
+--------+--------+



In [35]:
df_spark.createOrReplaceTempView('occupancy_sensor_data')

In [36]:
spark.sql('select * from occupancy_sensor_data').show()

+-----+-------------------+-----------+----------------+-----+----------------+-------------------+---------+
|index|               date|Temperature|        Humidity|Light|             CO2|      HumidityRatio|Occupancy|
+-----+-------------------+-----------+----------------+-----+----------------+-------------------+---------+
|    1|2015-02-04 17:51:00|      23.18|          27.272|426.0|          721.25|0.00479298817650529|        1|
|    2|2015-02-04 17:51:59|      23.15|         27.2675|429.5|           714.0|0.00478344094931065|        1|
|    3|2015-02-04 17:53:00|      23.15|          27.245|426.0|           713.5|0.00477946352442199|        1|
|    4|2015-02-04 17:54:00|      23.15|            27.2|426.0|          708.25|0.00477150882608175|        1|
|    5|2015-02-04 17:55:00|       23.1|            27.2|426.0|           704.5|0.00475699293331518|        1|
|    6|2015-02-04 17:55:59|       23.1|            27.2|419.0|           701.0|0.00475699293331518|        1|
|    7|201

In [45]:
spark.sql('select temperature, humidity, occupancy from occupancy_sensor_data where occupancy == 1 and temperature>20 and humidity>25').show()

+-----------+----------------+---------+
|temperature|        humidity|occupancy|
+-----------+----------------+---------+
|      23.18|          27.272|        1|
|      23.15|         27.2675|        1|
|      23.15|          27.245|        1|
|      23.15|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|     23.075|          27.175|        1|
|     23.075|           27.15|        1|
|       23.1|            27.1|        1|
|       23.1|27.1666666666667|        1|
|      23.05|           27.15|        1|
|       23.0|          27.125|        1|
|       23.0|          27.125|        1|
|      21.84|         25.0225|        1|
|      21.89|           25.15|        1|
|      21.89|            25.2|        1|
|      21.89|            25.2|        1|
+-----------+----------------+---------+
only showing top

In [44]:
df_spark.filter((df_spark['Temperature']>20) & (df_spark['humidity']>25) & (df_spark['occupancy']==1)).select(['temperature','humidity','occupancy']).show()

+-----------+----------------+---------+
|temperature|        humidity|occupancy|
+-----------+----------------+---------+
|      23.18|          27.272|        1|
|      23.15|         27.2675|        1|
|      23.15|          27.245|        1|
|      23.15|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|       23.1|            27.2|        1|
|     23.075|          27.175|        1|
|     23.075|           27.15|        1|
|       23.1|            27.1|        1|
|       23.1|27.1666666666667|        1|
|      23.05|           27.15|        1|
|       23.0|          27.125|        1|
|       23.0|          27.125|        1|
|      21.84|         25.0225|        1|
|      21.89|           25.15|        1|
|      21.89|            25.2|        1|
|      21.89|            25.2|        1|
+-----------+----------------+---------+
only showing top

In [46]:
df_spark.agg({'temperature':'max','humidity':'max'}).show()

+----------------+-------------+
|max(temperature)|max(humidity)|
+----------------+-------------+
|           23.18|      39.1175|
+----------------+-------------+



In [58]:
df_spark.agg({'temperature':'stddev','humidity':'min','humidity':'max'}).show()

+-------------------+-------------+
|stddev(temperature)|max(humidity)|
+-------------------+-------------+
| 1.0169164411058336|      39.1175|
+-------------------+-------------+



In [64]:
spark.sql('select max(temperature) as max_temp,avg(temperature),min(temperature),max(humidity),avg(humidity),min(humidity) from occupancy_sensor_data' ).show()

+--------+-----------------+----------------+-------------+------------------+-------------+
|max_temp| avg(temperature)|min(temperature)|max(humidity)|     avg(humidity)|min(humidity)|
+--------+-----------------+----------------+-------------+------------------+-------------+
|   23.18|20.61908364034621|            19.0|      39.1175|25.731507286422527|       16.745|
+--------+-----------------+----------------+-------------+------------------+-------------+



In [66]:
spark.sql('select max(temperature) from occupancy_sensor_data group by humidity').show()

+----------------+
|max(temperature)|
+----------------+
|            22.7|
|           20.79|
|            22.1|
|19.6333333333333|
|         21.4175|
|          20.125|
|          20.445|
|            21.5|
|            21.7|
|            22.6|
|           21.65|
|         21.7675|
|            22.6|
|            19.7|
|            19.5|
|          20.675|
|           21.39|
|           22.23|
|            21.2|
|         22.9725|
+----------------+
only showing top 20 rows



# Question 2 (DCHB_Village_Amenities_Bangalore.csv)

In [10]:
df_spark = spark.read.csv('DCHB_Village_Amenities_Bangalore.csv',header=True,inferSchema=True)

In [11]:
print((df_spark.count(),len(df_spark.columns)))

(588, 396)


# 2.1 Find out list of villages which are having a town within 10 km

In [12]:
import pandas as pd

In [13]:
df_spark.columns

['State Code',
 'State Name',
 'District Code',
 'District Name',
 'Sub District Code',
 'Sub District Name',
 'Village Code',
 'Village Name',
 'CD Block Code',
 'CD Block Name',
 'Gram Panchayat Code',
 'Gram Panchayat Name',
 'Reference Year',
 'Sub District Head Quarter (Name)',
 'Sub District Head Quarter (Distance in km)',
 'District Head Quarter (Name)',
 'District Head Quarter (Distance in km)',
 'Nearest Statutory Town (Name)',
 'Nearest Statutory Town (Distance in km)',
 'Within the State/UT (Name)',
 'Within the State/UT (Distance in km)',
 'Outside the State/UT, if nearest one is not within the State/UT (Name)',
 'Outside the State/UT distance, if nearest one is not within the State/UT (Distance in km)',
 'Total Geographical Area (in Hectares)',
 'Total  Households ',
 'Total Population of Village',
 'Total Male Population of Village',
 'Total Female Population of Village',
 'Total Scheduled Castes Population of Village',
 'Total Scheduled Castes Male Population of Village'

In [14]:
#df_spark.columns=[i.replace(' ','_').strip() for i in df_spark.columns]

In [15]:
un=df_spark.withColumnRenamed('Nearest Town Distance from Village (in Km.)','NearVillage')

In [16]:
un = un.withColumnRenamed("Village Name","Village_Name")

In [17]:
un.select(["Village_Name"]).where(un["NearVillage"] <= 10).show()

+-------------------+
|       Village_Name|
+-------------------+
|         Gopalapura|
|Shamabhattara Palya|
|           Totagere|
|    Bommasettihalli|
|       Syadamipalya|
|        Bethanagere|
|           Rampalya|
|             Huskur|
|    Hosahalli Palya|
|        Govindapura|
|        Honnasandra|
|      Huchana Palya|
|         Pillahalli|
|            Nagarur|
|          Kodipalya|
|Seshagiri Rao Palya|
|         Mathahalli|
|         Narasipura|
|      Muniyanapalya|
|               Alur|
+-------------------+
only showing top 20 rows



# 2.2 Find out the total number of village which are having a town within 10 km

In [18]:
un=un.withColumnRenamed('Primary Health Centre (Numbers)','Primary_Health_Centre_(Numbers)')

In [19]:
from pyspark.sql.functions import countDistinct

In [20]:
un.agg(countDistinct('Primary_Health_Centre_(Numbers)')).show()

+--------------------------------------+
|count(Primary_Health_Centre_(Numbers))|
+--------------------------------------+
|                                     3|
+--------------------------------------+



# 2.3 Using SQL query, find out the number of villages with cultivate paddy